In [1]:
# Necessary to import custom modules
import os
from pathlib import Path
os.chdir("/home/jovyan/work")

import pandas as pd
import numpy as np
from datetime import datetime


import geopy.distance
import mpu


from src.features import get_feature

In [2]:
#!pip install geopy
!pip install mpu 

You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Distance traveled so far and days since last game

In [3]:
df = get_feature("list_all_games_with_location")

With this we can get the distance a team traveled till the date we predict a match at and we can also get the days since its last game. 

## Distance traveled so far

TODO: eventually the calculation has to be adapted because sometimes teams might not travel home after an away game but instead directly to the next away game. So far the distance to an away game is multiplied with 2! 

In [4]:
df['lat'] = df['lat'].convert_objects(convert_numeric=True)
df['long'] = df['long'].convert_objects(convert_numeric=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [11]:
#distance calculation of teams 

distance_to_arena_and_back = [1]
for x in df.index.values[1:-1]:
    if x == 0:
        distance_to_arena_and_back.append(0)
    elif all([df.team[x] == df.team[x-1], df.Season[x] == df.Season[x-1]]):
        distance = mpu.haversine_distance((df.lat[x], df.long[x]), (df.lat[x+1], df.long[x+1]))
        distance_to_arena_and_back.append(distance)
    else:
        distance_to_arena_and_back.append(1)
distance_to_arena_and_back.append(1)

#df["distance_to_arena_and_back"] = [x * 2 for x in distance_to_arena_and_back]
df["distance_to_arena"] = distance_to_arena_and_back

In [12]:
distance_traveled = [1]
for x in df.index.values[1:-1]:
    if all([df.distance_to_arena_and_back[x] != 2]):
        temp = df.distance_to_arena_and_back[x] + distance_traveled[x-1]
        distance_traveled.append(temp)
    else:
        distance_traveled.append(1)
distance_traveled.append(1)
df["distance_traveled"] = distance_traveled

In [13]:
df.head(100)

,date,team,game_id,Season,Arenaname,long,lat,distance_to_arena_and_back,distance_traveled,distance_to_arena
0,1.445990e+09,Atlanta Hawks,31176,2015/2016,Philips Arena,-84.396389,33.757222,1.000000,1.000000,1.000000
1,1.446163e+09,Atlanta Hawks,31195,2015/2016,Madison Square Garden,-73.993611,40.750556,1203.611524,1204.611524,1203.611524
2,1.446250e+09,Atlanta Hawks,31196,2015/2016,Philips Arena,-84.396389,33.757222,364.550846,1569.162371,364.550846
3,1.446404e+09,Atlanta Hawks,31216,2015/2016,Spectrum Center,-80.839167,35.225000,1051.924282,2621.086653,1051.924282
4,1.446597e+09,Atlanta Hawks,31232,2015/2016,American Airlines Arena,-80.188056,25.781389,975.185519,3596.272172,975.185519
5,1.446685e+09,Atlanta Hawks,31235,2015/2016,Philips Arena,-84.396389,33.757222,683.663527,4279.935698,683.663527
6,1.446858e+09,Atlanta Hawks,31255,2015/2016,Smoothie King Center,-90.081944,29.948889,683.663527,4963.599225,683.663527
7,1.446943e+09,Atlanta Hawks,31260,2015/2016,Philips Arena,-84.396389,33.757222,0.000000,4963.599225,0.000000
8,1.447117e+09,Atlanta Hawks,31274,2015/2016,Philips Arena,-84.396389,33.757222,0.000000,4963.599225,0.000000
9,1.447290e+09,Atlanta Hawks,31288,2015/2016,Philips Arena,-84.396389,33.757222,1506.507272,6470.106497,1506.507272


## Days since last game 

In [9]:
df["date2"] = [datetime.fromtimestamp(x) for x in df.date]

In [10]:
days_since_last_game = [0]
for x in df.index.values[1:-1]:
    if df.team[x] == df.team[x-1]:
        days = df.date2[x] - df.date2[x-1] 
        days_since_last_game.append(days)
    else:
        days_since_last_game.append(0)
days_since_last_game.append(0)

df["days_since_last_game"] = days_since_last_game



In [11]:
df.head(100)

,date,team,game_id,Season,Arenaname,long,lat,distance_to_arena_and_back,distance_traveled,date2,days_since_last_game
0,1.445990e+09,Atlanta Hawks,31176,2015/2016,Philips Arena,-84.396389,33.757222,2.000000,1.000000,2015-10-28 00:00:00,0
1,1.446163e+09,Atlanta Hawks,31195,2015/2016,Madison Square Garden,-73.993611,40.750556,2407.223049,2408.223049,2015-10-30 00:00:00,2 days 00:00:00
2,1.446250e+09,Atlanta Hawks,31196,2015/2016,Philips Arena,-84.396389,33.757222,729.101693,3137.324741,2015-10-31 00:00:00,1 days 00:00:00
3,1.446404e+09,Atlanta Hawks,31216,2015/2016,Spectrum Center,-80.839167,35.225000,2103.848565,5241.173306,2015-11-01 19:00:00,1 days 19:00:00
4,1.446597e+09,Atlanta Hawks,31232,2015/2016,American Airlines Arena,-80.188056,25.781389,1950.371037,7191.544343,2015-11-04 00:30:00,2 days 05:30:00
5,1.446685e+09,Atlanta Hawks,31235,2015/2016,Philips Arena,-84.396389,33.757222,1367.327053,8558.871396,2015-11-05 01:00:00,1 days 00:30:00
6,1.446858e+09,Atlanta Hawks,31255,2015/2016,Smoothie King Center,-90.081944,29.948889,1367.327053,9926.198450,2015-11-07 01:00:00,2 days 00:00:00
7,1.446943e+09,Atlanta Hawks,31260,2015/2016,Philips Arena,-84.396389,33.757222,0.000000,9926.198450,2015-11-08 00:30:00,0 days 23:30:00
8,1.447117e+09,Atlanta Hawks,31274,2015/2016,Philips Arena,-84.396389,33.757222,0.000000,9926.198450,2015-11-10 01:00:00,2 days 00:30:00
9,1.447290e+09,Atlanta Hawks,31288,2015/2016,Philips Arena,-84.396389,33.757222,3013.014544,12939.212994,2015-11-12 01:00:00,2 days 00:00:00


# Calculating PER

In [28]:
df = get_feature("PER")

PER is not a proper feature name
